> First time use: follow instructions in the README.md file in this directory.

# **[PT]** Título


Para criar uma versão `html` sem o código fonte, fazer, na directoria "notebooks":

    jupyter nbconvert students_overview.ipynb --to html --no-input


# **[EN]** Title

_To create a `html` version without the code cells, do, in the "notebooks" directory_:

    jupyter nbconvert remissivas.ipynb --to html --no-input

## Setup

In [4]:
from timelinknb import current_time,current_machine, get_mhk_db
import ucalumni.config as alumniconf

db_name = alumniconf.mhk_db_name
db = get_mhk_db(db_name)
print(current_machine,current_time,f'db={db_name}')

mini-m1.local 2022-04-30 14:49:13.989724 db=ucalumni


## **[PT]** Naturalidades

In [38]:
from timelinknb.pandas import attribute_values


# Get list of people with attribute nome-vide and add naturalidade and faculdade
naturalidades = attribute_values('naturalidade')

In [39]:
naturalidades[naturalidades['count']<3].sort_values('count', ascending=False).head(40)

,count
value,
Ervas Tenras,2
Ronfe,2
Carvalhais de Baixo,2
Carvalhaes,2
Luzinde,2
Romarigais do Coura,2
"Roma, Itália",2
Maçal do Chão,2
"Carriça, Viseu",2


## Usar Toponímia das cartas militares

In [41]:
import pandas as pd
toponimia = pd.read_csv('../extras/toponimia/toponima-militar.csv')
toponimia['nome_concelho'] = toponimia['nome']+", "+toponimia['concelho']
toponimia.head()

,nome,codigoine,xx,yy,cartamilitar,tipo,concelho,distrito,obs,id,nome_concelho
0,ABEGOARIA,140104,195981,258102,355,VG,Abrantes,Santarém,(null),1,"ABEGOARIA, Abrantes"
1,Abegoaria de Baixo,140104,195169,256737,355,CAS,Abrantes,Santarém,(null),2,"Abegoaria de Baixo, Abrantes"
2,Abegoaria de Cima,140104,195639,256644,355,CAS,Abrantes,Santarém,(null),3,"Abegoaria de Cima, Abrantes"
3,ABITUREIRA,140103,212055,273288,332,VG,Abrantes,Santarém,(null),4,"ABITUREIRA, Abrantes"
4,Abrançalha de Baixo,140113,191736,278390,331,POVI,Abrantes,Santarém,(null),5,"Abrançalha de Baixo, Abrantes"


In [27]:
toponimia[toponimia.nome == "Lisboa"]

,nome,codigoine,xx,yy,cartamilitar,tipo,concelho,distrito,obs,id
64490,Lisboa,110624,112483,194793,431,SC,Lisboa,Lisboa,(null),64491


In [36]:
for local in naturalidades.index.values[:100]:
    clocal = local.lstrip("?-/").rstrip("?-/").strip()
    topo_list = toponimia.loc[toponimia.nome == local].index.values
    if len(topo_list) == 0:
        print(f"|{clocal}| not found")
    else:
        for t in topo_list:
            toponimo = toponimia.loc[[t]].iloc[0]
            print(clocal,toponimo['nome'],toponimo['concelho'],toponimo['distrito'])

|"""
                mostrou que seu pai se chamava Manuel Ribeiro e por ordem do mesmo senhor se lhe passaram as cartas com o nome de seu pai Manuel Ribeiro e fiz esta declaração para que não faça dúvida o assento das matrículas
            """| not found
|(Estado de Minas Gerais)Brasil| not found
|Lisboa| not found
|Vila Franca| not found
|Lisboa| not found
|23031686| not found
|28111637| not found
|| not found
|do Campo| not found
|Inglesa| not found
|Lisboa| not found
|a| not found
A de Barros A de Barros Sernancelhe Viseu
|A dos Francos| not found
|a- Velha| not found
|Abaças| not found
Abade Abade Chamusca Santarém
|Abade de São Romão| not found
Abadia Abadia Amares Braga
Abadia Abadia Covilhã Castelo Branco
Abadia Abadia Leiria Leiria
Abadia Abadia Mafra Lisboa
Abadia Abadia Mangualde Viseu
Abadia Abadia Mêda Guarda
Abadia Abadia Miranda do Douro Bragança
Abadia Abadia Ourém Santarém
Abadia Abadia Pinhel Guarda
Abadia Abadia Sobral de Monte Agraço Lisboa
Abadia Abadia Tomar Sant

## **[PT]** Contagens